In [1]:
import pandas as pd
import os

In [2]:
file_dir = os.path.join('train_data')

file_path = os.path.join(file_dir,'train_data.csv')

df = pd.read_csv(file_path)

print(df)

      Molecule ChEMBL ID Standard Type Standard Relation  Standard Value  \
0          CHEMBL4443947          IC50               '='           0.022   
1          CHEMBL4556091          IC50               '='           0.026   
2          CHEMBL4566431          IC50               '='           0.078   
3          CHEMBL4545898          IC50               '='           0.081   
4          CHEMBL4448950          IC50               '='           0.099   
...                  ...           ...               ...             ...   
54583      CHEMBL5201128          IC50               '='           4.000   
54584      CHEMBL4080062          IC50               '='           4.600   
54585        CHEMBL75232          IC50               '='          55.000   
54586      CHEMBL5288744          IC50               '='           3.000   
54587      CHEMBL5078078          IC50               '>'       10000.000   

      Standard Units  pChEMBL Value Assay ChEMBL ID Target ChEMBL ID  \
0              

In [3]:
df = df[df['Standard Units'] == 'nM'] # nM이 아닌 행 제거
df = df[df['Standard Type'] == 'IC50'] # nM이 아닌 행 제거
df = df[df['Target Organism'] == 'Homo sapiens'] # Homo sapines가 아닌 행 제거
df = df[df['Standard Relation'] == "'='"] # '='이 아닌 행 제거
df = df[df['Target Type'] == 'SINGLE PROTEIN'] # 'SINGLE PROTEIN'이 아닌 행 제거
df = df.dropna(subset=['Standard Value']) # Standard Value 비어있는 행 제거

# 인덱스 재설정
df.reset_index(drop=True, inplace=True)

print(df.shape)
print(df)

(43514, 15)
      Molecule ChEMBL ID Standard Type Standard Relation  Standard Value  \
0          CHEMBL4443947          IC50               '='           0.022   
1          CHEMBL4556091          IC50               '='           0.026   
2          CHEMBL4566431          IC50               '='           0.078   
3          CHEMBL4545898          IC50               '='           0.081   
4          CHEMBL4448950          IC50               '='           0.099   
...                  ...           ...               ...             ...   
43509      CHEMBL5180534          IC50               '='         158.000   
43510      CHEMBL5201128          IC50               '='           4.000   
43511      CHEMBL4080062          IC50               '='           4.600   
43512        CHEMBL75232          IC50               '='          55.000   
43513      CHEMBL5288744          IC50               '='           3.000   

      Standard Units  pChEMBL Value Assay ChEMBL ID Target ChEMBL ID  \
0  

In [5]:
# 제거할 열 목록
columns_to_drop = ['Molecule ChEMBL ID','Standard Type', 'Standard Relation', 'Standard Units', 
                   'pChEMBL Value','Assay ChEMBL ID', 'Target ChEMBL ID', 'Target Organism', 'Target Type',
                   'Document ChEMBL ID', 'IC50_nM', 'pIC50']

# 지정된 열 제거
df = df.drop(columns=columns_to_drop) # columns_to_drop이 아닌 행 거거

print(df.shape)
print(df)


(43514, 3)
       Standard Value                                    Target Name  \
0               0.022     Interleukin-1 receptor-associated kinase 4   
1               0.026     Interleukin-1 receptor-associated kinase 4   
2               0.078     Interleukin-1 receptor-associated kinase 4   
3               0.081     Interleukin-1 receptor-associated kinase 4   
4               0.099     Interleukin-1 receptor-associated kinase 4   
...               ...                                            ...   
43509         158.000  Vascular endothelial growth factor receptor 2   
43510           4.000  Vascular endothelial growth factor receptor 2   
43511           4.600  Vascular endothelial growth factor receptor 2   
43512          55.000  Vascular endothelial growth factor receptor 2   
43513           3.000  Vascular endothelial growth factor receptor 2   

                                                  Smiles  
0      CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...  
1     

In [7]:
new_column_order = ['Smiles', 'Target Name', 'Standard Value']
df = df[new_column_order]

# 결과 확인
print(df)

                                                  Smiles  \
0      CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...   
1      CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...   
2      CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...   
3      CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...   
4      COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...   
...                                                  ...   
43509  COc1cc(CCc2cc(-c3nc4ccc(CN5C[C@@H](C)N[C@@H](C...   
43510  COc1cc2nccc(Oc3ccc(NC(=O)C4(C(=O)NC5CC5)CC4)c(...   
43511  O=C(Nc1cccc(C(F)(F)F)c1)c1cccc2cc(Oc3cc(CO)ncn...   
43512  Clc1ccc(Nc2nnc(Cc3ccncc3)c3ccccc23)cc1.O=C(O)C...   
43513  Nc1cc(Oc2ccc3c(C(=O)Nc4cccc(C(F)(F)F)c4)cccc3c...   

                                         Target Name  Standard Value  
0         Interleukin-1 receptor-associated kinase 4           0.022  
1         Interleukin-1 receptor-associated kinase 4           0.026  
2         Interleukin-1 receptor-associated kinase 4           0.0

In [11]:
df.to_csv('train_data/regression_train_data.csv', index = False)